In [1]:
%pip install feedparser
%pip install pandas
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import feedparser
from pandas import json_normalize
import pandas as pd
import requests
import re

rss_url='https://www.amnesty.org/en/feed/'
#Read feed xml data
news_feed = feedparser.parse(rss_url)

#Flatten data
df_news_feed=json_normalize(news_feed.entries)

# Function to remove post info from summaries
def remove_post_info(text):
    return re.sub(r'<p>The post.*</p>', '', text)

# Function to remove HTML tags from a single text
def remove_html_tags(text):
    return re.sub(r'<[^>]*>', '', text)

#Read articles' titles
#series = df_news_feed.title

#For summary, use the following code
df_news_feed.summary = df_news_feed.summary.apply(remove_post_info)
series = df_news_feed.summary.apply(remove_html_tags)

#Convert the pandas.core.series.Series into a list
conjunt = series.tolist()


In [7]:
# Print each item from the 'conjunt' list on a new line

for item in conjunt:
    print(item)


UN Member states must use the UN’s upcoming Universal Periodic Review (UPR) on Bangladesh to hold the authorities to account for the gross human rights violations and rapidly deteriorating human rights situation in the country ahead of its general elections, said Amnesty International today. “Bangladesh’s fourth UPR is taking place at a time when human [&#8230;]

More than a million signatures from people around the world have been gathered for an Amnesty International petition demanding an immediate ceasefire to end the unparalleled escalation of hostilities in Gaza and Israel since 7 October 2023. In response, Erika Guevara-Rosas, Senior Director of Research, Advocacy, Policy and Campaigns at Amnesty International, said: “The world [&#8230;]

The Government of Pakistan must immediately halt the continued detentions, deportations and widespread harassment of Afghan refugees, said Amnesty International today. “Thousands of Afghan refugees are being used as political pawns to be returne

In [32]:
from transformers import pipeline


In [47]:
# This model is a `zero-shot-classification` model.
# It will classify text, except you are free to choose any label you might imagine
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [48]:
# Assigning the value of 'conjunt' to 'sequence', used in the classifier object.
sequence = conjunt

# Opening the file "labels.txt" in read mode with UTF-8 encoding
# and printing the first three labels.

label_file = open("labels.txt", "r", encoding='utf-8-sig')
labels = label_file.read().split(',')
print(labels[0:3])

['Censorship and Freedom of Expression', ' Freedom of Movement', ' Internet and Social Media']


In [49]:
# Predictions on the given sequence, with multiple labels.
# With Bart-large, if sequence = summary, it taks aprox. 5 minutes.
output = classifier(sequence,labels,multi_label=True)


In [50]:
# To know the type of the output --> a list.
type(output)

list

In [52]:
# Printing the results for sequence 1 from Bart classifier.
for list_item in output[:1]:
    for key, value in list_item.items():
        print(key, value)

sequence UN Member states must use the UN’s upcoming Universal Periodic Review (UPR) on Bangladesh to hold the authorities to account for the gross human rights violations and rapidly deteriorating human rights situation in the country ahead of its general elections, said Amnesty International today. “Bangladesh’s fourth UPR is taking place at a time when human [&#8230;]

labels [' UN', ' International Justice', ' Unlawful Detention', ' Unfair Trials', ' Youth and Human Rights', ' Protests and Demonstrations', ' Killings and Disappearances', ' Prisoners of Conscience', ' Impunity', ' Technology and Human Rights', ' Human Rights Defenders and Activists', ' Discrimination', ' Disappearances', ' Unlawful Killings', 'Censorship and Freedom of Expression', ' Older People', ' Torture and other ill-treatment', ' Freedom of Movement', ' Slums and the Right to Housing', ' Detention', ' Internally Displaced People', ' Debt and Human Rights', ' Freedom of Association', ' Strategic Litigation', ' 

In [53]:
# Function to get the top 'n' labels and scores
def get_top_n_labels_and_scores(data_item, n):
    label_scores = dict(zip(data_item['labels'], data_item['scores']))
    top_scores = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)[:n]
    return data_item['sequence'], top_scores

# Get the top 'n' labels and scores for each data item and print it
i = 1  # Initialize the counter, to print only the first item. Comment otherwise.
for i, item in enumerate(output, 1):
    sequence, top_scores = get_top_n_labels_and_scores(item, 5)
    print(f"Sequence {i}: {sequence}")
    for label, score in top_scores:
        print(f"Label: {label}, Score: {score}")
    print()
    break  # Break out of the loop after processing the first item. Comment otherwise.

# Output file path
output_file_path = 'output.txt'

# Get the top 'n' labels and scores for each data item and save it to a file
with open(output_file_path, 'w') as output_file:
    for i, item in enumerate(output, 1):
        sequence, top_scores = get_top_n_labels_and_scores(item, 5)
        output_file.write(f"Sequence {i}: {sequence}\n")
        for label, score in top_scores:
            output_file.write(f"Label: {label}, Score: {score}\n")
        output_file.write('\n')

# Print a message indicating the successful save
print(f"Output saved to {output_file_path}")

Sequence 1: UN Member states must use the UN’s upcoming Universal Periodic Review (UPR) on Bangladesh to hold the authorities to account for the gross human rights violations and rapidly deteriorating human rights situation in the country ahead of its general elections, said Amnesty International today. “Bangladesh’s fourth UPR is taking place at a time when human [&#8230;]

Label:  UN, Score: 0.93719482421875
Label:  International Justice, Score: 0.731397271156311
Label:  Unlawful Detention, Score: 0.45114412903785706
Label:  Unfair Trials, Score: 0.44957607984542847
Label:  Youth and Human Rights, Score: 0.4292548894882202

Output saved to output.txt
